# DC Line dispatch with pandapower OPF
This is an introduction into the usage of the pandapower optimal power flow with dc lines.

## Example Network

We use the following four bus example network for this tutorial:

<img src="pics/example_opf_dcline.png" width="100%">

We first create this network in pandapower:

In [ ]:
from pandapower.create import (
    create_empty_network,
    create_bus,
    create_lines,
    create_dcline,
    create_ext_grid,
    create_load,
    create_poly_cost
)
from pandapower.run import runpp, runopp

In [ ]:
net = create_empty_network()

b1 = create_bus(net, 380)
b2 = create_bus(net, 380)
b3 = create_bus(net, 380)
b4 = create_bus(net, 380)
b5 = create_bus(net, 380)

l1, l2, l3 = create_lines(net, [b1, b3, b4], [b2, b4, b5], [30, 20, 20], "490-AL1/64-ST1A 380.0")

dcl1 = create_dcline(net, name="dc line", from_bus=b2, to_bus=b3, p_mw=200, loss_percent=1.0,
                  loss_mw=0.5, vm_from_pu=1.01, vm_to_pu=1.012, max_p_mw=1000,
                  in_service=True)

eg1 = create_ext_grid(net, b1, 1.02, min_p_mw=0.)
eg2 = create_ext_grid(net, b5, 1.02, min_p_mw=0.)

l1 = create_load(net, bus=b4, p_mw=800, controllable = False)

We now run a regular load flow to check out the DC line model:

In [ ]:
runpp(net)

The transmission power of the DC line is defined in the loadflow as given by the p_kw parameter, which was set to 200 MW:

In [ ]:
net.res_dcline

The losses amount to 2.5 MW, which are made up of 0.5 MW conversion loss and 200 MW * 0.01 = 2 MW transmission losses. The voltage setpoints defined at from and to bus are complied with. 

Now lets define costs for the external grids to run an OPF:

In [ ]:
costeg0 = create_poly_cost(net, 0, 'ext_grid', cp1_eur_per_mw=10)
costeg1 = create_poly_cost(net, 1, 'ext_grid', cp1_eur_per_mw=8)
net.bus['max_vm_pu'] = 1.5
net.line['max_loading_percent'] = 1000

In [ ]:
runopp(net, delta=1e-16)

This function runs an Optimal Power Flow using the PYPOWER OPF. To make sure that the PYPOWER OPF converges, we decrease the power tolerance `delta` (the default value is `delta=1e-10`). The power tolerance `delta` is a measure of the extent to which exceeding of minimum and maximum power limits is tolerated. That is, in above case, the limits considered by the OPF for the generators are `min_p_mw - delta` and `max_p_mw + delta` as lower and upper bound respectively on the active power. 

Since we defined lower costs for Ext Grid 2, it fully services the load:

In [ ]:
net.res_ext_grid

While the DC line does not transmit any power:

In [ ]:
net.res_dcline

If we set the costs of the left grid to a lower value than the right grid and run the loadflow again:

In [ ]:
net.poly_cost.cp1_eur_per_mw.at[costeg0] = 8
net.poly_cost.cp1_eur_per_mw.at[costeg1] = 10
runopp(net, delta=1e-16)

We can see that the power now comes from the left ext_grid:

In [ ]:
net.res_ext_grid

And is transmitted over the DC line:

In [ ]:
net.res_dcline

We can however see that the lines on the left hand side are now overloaded:

In [ ]:
net.res_line.loading_percent

If we set the maximum line loading to 100% and run the OPF again:

In [ ]:
net.line["max_loading_percent"] = 100
runopp(net, delta=1e-16)

We can see that the lines are no longer overloaded:

In [ ]:
net.res_line.loading_percent

Because the load is serviced from both grids:

In [ ]:
net.res_ext_grid

And the DC line transmits only part of the power needed to service the load:

In [ ]:
net.res_dcline

Finally, we can also define transmission costs for the DC line:

In [ ]:
costeg1 = create_poly_cost(net, 0, 'dcline', cp1_eur_per_mw=3)
runopp(net, delta=1e-16)

Because the sum of the costs for generating power on the left hand side (8) and transmitting it to the right side (3) is now larger than for generating on the right side (10), the OPF draws as much power from the right side as is possible without violating line loading constraints:

In [ ]:
net.res_line.loading_percent

In [ ]:
net.res_dcline

If we relax the line loading constraint and run the OPF again:

In [ ]:
net.line["max_loading_percent"] = 1000
runopp(net, delta=1e-16)

The load is once again fully serviced by the grid on the right hand side:

In [ ]:
net.res_ext_grid

And the DC line is in open loop operation:

In [ ]:
net.res_dcline

Little consistency check:

In [ ]:
net.res_ext_grid.p_mw.at[0]*8 + net.res_ext_grid.p_mw.at[1]*10 + net.res_dcline.p_from_mw.at[0]*3

In [ ]:
net.res_cost